In [ ]:
import pandas as pd
import numpy as np

In [ ]:
file_data = pd.read_csv("data.csv",encoding='latin1')
file_data.head()

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (2,6,7,9,10,11,12,13,15,16,17) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Sl. No.,YEAR,MONTH,DATE,ORIGIN TIME,Unnamed: 5,MAGNITUDE,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,LAT (N),LONG (E),DEPTH (km),INTENSITY,Unnamed: 16,Unnamed: 17,LOCATION,REFERENCE
0,NaN,NaN,NaN,NaN,(UTC),(IST),Mw,Mw,Mb,Mb,Ms,ML,NaN,NaN,NaN,MM,MMI,MME,NaN,NaN
1,1.0,-2474.0,0,0.0,NaN,NaN,7.5,7.5,NaN,6.969202899,7.260619977,7.427072403,71,24,0.0,NaN,NaN,NaN,NaN,Dr STGR
2,2.0,-325.0,0,0.0,NaN,NaN,7.5,7.5,NaN,6.969202899,7.260619977,7.427072403,71,24,0.0,NaN,NaN,NaN,NaN,Dr STGR
3,3.0,25.0,0,0.0,NaN,NaN,7.5,7.5,NaN,6.969202899,7.260619977,7.427072403,72.9,33.72,0.0,NaN,NaN,NaN,NaN,Dr STGR
4,4.0,26.0,5,10.0,08.19.10.0,NaN,6.1397,6.1397,5.737047101,5.737047101,5.698851894,5.999685205,17.3,80.1,NaN,NaN,NaN,NaN,NaN,NEIC


In [ ]:
# drop row 0
file_data = file_data.drop(file_data.index[0])
file_data.head()

,Sl. No.,YEAR,MONTH,DATE,ORIGIN TIME,Unnamed: 5,MAGNITUDE,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,LAT (N),LONG (E),DEPTH (km),INTENSITY,Unnamed: 16,Unnamed: 17,LOCATION,REFERENCE
1,1.0,-2474.0,0,0.0,NaN,NaN,7.5,7.5,NaN,6.969202899,7.260619977,7.427072403,71,24,0.0,NaN,NaN,NaN,NaN,Dr STGR
2,2.0,-325.0,0,0.0,NaN,NaN,7.5,7.5,NaN,6.969202899,7.260619977,7.427072403,71,24,0.0,NaN,NaN,NaN,NaN,Dr STGR
3,3.0,25.0,0,0.0,NaN,NaN,7.5,7.5,NaN,6.969202899,7.260619977,7.427072403,72.9,33.72,0.0,NaN,NaN,NaN,NaN,Dr STGR
4,4.0,26.0,5,10.0,08.19.10.0,NaN,6.1397,6.1397,5.737047101,5.737047101,5.698851894,5.999685205,17.3,80.1,NaN,NaN,NaN,NaN,NaN,NEIC
5,5.0,26.0,5,10.0,08.19.10.0,NaN,6.1397,6.1397,5.737047101,5.737047101,6.075520196,5.999685205,26,97,80.0,NaN,NaN,NaN,Myanmar,G-R


In [ ]:
# shuffle rows
file_data = file_data.sample(frac = 1) 
file_data.tail()

,Sl. No.,YEAR,MONTH,DATE,ORIGIN TIME,Unnamed: 5,MAGNITUDE,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,LAT (N),LONG (E),DEPTH (km),INTENSITY,Unnamed: 16,Unnamed: 17,LOCATION,REFERENCE
39757,39742.0,2005.0,10,13.0,134506.45,NaN,NaN,3.5596,3.4,3.4,2.73662,3.29234,34.54,73.67,10.0,NaN,NaN,NaN,NaN,NEIC / USGS
23981,23966.0,1997.0,09,21.0,NaN,NaN,4.0,4.0,NaN,3.798913043,3.242250287,3.754459601,78.57,31.9,0.0,NaN,NaN,NaN,NaN,Dr STGR
21516,21501.0,1996.0,03,23.0,NaN,NaN,4.2,4.2,NaN,3.980072464,3.471871412,3.96432319,70.17,36.49,223.2,NaN,NaN,NaN,NaN,Dr STGR
31943,31928.0,2002.0,12,25.0,NaN,NaN,4.2,4.2,NaN,3.980072464,3.471871412,3.96432319,69.35,37.82,0.0,NaN,NaN,NaN,NaN,Dr STGR
39840,39825.0,2005.0,10,19.0,124727.32,NaN,NaN,5.326,5,5,5.07956,5.14586,34.52,73.05,10.0,NaN,NaN,NaN,NaN,NEIC / USGS


In [ ]:
# file_data.columns
# filter data
req_features = ['LAT (N)','LONG (E)','DEPTH (km)','MAGNITUDE']
filtered_data = file_data[req_features]
print(filtered_data.shape)
x_features = ['LAT (N)','LONG (E)','DEPTH (km)']
X = filtered_data[x_features]
y = filtered_data['MAGNITUDE']
print(X.shape)
print(y.shape)

(52989, 4)
(52989, 3)
(52989,)


In [ ]:
import re
def edit_latitude(val):
  if type(val) == str:
    new_val = val.replace('?','').replace(' ','').replace(',','.')
    new_val = re.sub('[^a-zA-Z0-9\.]', '', new_val)
    if new_val[-1] == 'N':
        new_val = float(new_val[0:-1])
    elif new_val[-1] == 'S':
        new_val = 180 -1*float(new_val[0:-1])
    else:
      new_val = float(new_val)  
    return new_val
  return float(val)

In [ ]:
# X.tail()
X['LAT (N)'] = X['LAT (N)'].apply(edit_latitude)
X.tail()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,LAT (N),LONG (E),DEPTH (km)
39757,34.54,73.67,10.0
23981,78.57,31.9,0.0
21516,70.17,36.49,223.2
31943,69.35,37.82,0.0
39840,34.52,73.05,10.0


In [ ]:
# convert y from object to float
# print(y.dtype)
# def convertToFloat(val):
#   return float(val)
# y = y.apply(convertToFloat)
# y = y.astype('float32')
# print(y.dtype)
# y.tail()

In [ ]:
import re
def edit_longitude(val):
  if type(val) == str:
    new_val = val.replace('?','').replace(' ','').replace(',','.')
    new_val = re.sub('[^a-zA-Z0-9\.]', '', new_val)
    if new_val[-1] == 'E':
        new_val = float(new_val[0:-1])
    elif new_val[-1] == 'W':
        new_val = 360 - 1*float(new_val[0:-1])
    elif new_val[-1] == 'S':
        new_val = float(new_val[0:-1])
    else:
      new_val = float(new_val)  
    return new_val
  return float(val)

In [ ]:
X['LONG (E)'] = X['LONG (E)'].apply(edit_longitude)
X.tail()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,LAT (N),LONG (E),DEPTH (km)
31670,96.76,2.25,0.0
23687,71.06,35.59,0.0
17541,96.29,4.12,0.0
24845,66.30,29.10,10.0
26223,71.09,37.46,0.0


In [ ]:
# test-train split
from sklearn.model_selection import train_test_split
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.3,random_state=1)

In [ ]:
from sklearn.impute import SimpleImputer
my_imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
imputed_train_X = pd.DataFrame(my_imputer.fit_transform(train_X))
imputed_test_X = pd.DataFrame(my_imputer.transform(test_X))

imputed_train_X.columns = train_X.columns
imputed_test_X.columns = test_X.columns

In [ ]:
# # KNN
# from sklearn.neighbors import KNeighborsClassifier as KNNmodel
# knn = KNNmodel() 

# from sklearn.metrics import classification_report
# knn.fit(train_X, train_y)
# y_pred = knn.predict(test_X)
# print(classification_report(val_y, y_pred))
# roc_auc_score(val_y, y_pred)


In [ ]:
# Decision Tree
from sklearn.tree import DecisionTreeRegressor
model = DecisionTreeRegressor(max_leaf_nodes=5, random_state=1)
print(train_X.tail())
# model.fit(train_X, train_y)
# preds_val = model.predict(test_X)
# mae = mean_absolute_error(test_y, preds_val)

       LAT (N)  LONG (E)  DEPTH (km)
15356    96.19      4.65        33.0
7634     96.74      4.65       182.8
28832    10.46     92.88        33.0
8430     71.55     36.69         0.0
28342    32.96     77.88        33.0
